In [26]:
using DataFrames
using DataFramesMeta
myData1 = readtable("C:/Users/subha/Desktop/ML - HW 2/Data1.csv",header=true);
myData2 = readtable("C:/Users/subha/Desktop/ML - HW 2/Data2.csv",header=true);



# Algorithmic Approach to Linear Regression


In [2]:
function pairwise(X, threshold)
    xcorr =[]
    ycorr =[]
    (j,k)=size(X)
    for i = 1:j
        for j = 1:k
            if (abs(X[i,j]) >= threshold && i!=j)
                push!(xcorr,i)
                push!(ycorr,j)
            end
        end
    end
    xycorr = hcat(xcorr,ycorr)
    return xycorr
end


pairwise (generic function with 1 method)

In [3]:
function spitindices(xycorr,i)
    n = xycorr[i,1]
    m = xycorr[i,2]
    return n,m
end
    

spitindices (generic function with 1 method)

In [4]:
X = myData1[1:end-1]
Y = myData1[end]

using MLDataUtils
(train_X, train_Y), (test_X, test_Y) = splitobs(shuffleobs((X,Y)), at=.7)
(tr_X, tr_y), (vl_X, vl_y) = splitobs(shuffleobs((train_X, train_Y)), at=0.67)


tr_X = Matrix(tr_X)
vl_X = Matrix(vl_X)
train_X = Matrix(train_X)
test_X = Matrix(test_X)

75×15 Array{Float64,2}:
 3.0145   2.00206  5.21028  4.80966  …  5.10616  5.17664  2.91837   4.61628
 7.4166   3.95453  5.02839  4.70284     4.83294  4.53411  6.12326   4.61811
 5.44684  4.82233  5.01611  5.04528     4.89085  4.70273  3.95705   5.01098
 6.45016  3.33815  4.67588  4.55304     4.42254  4.45643  4.29816   4.29062
 6.49142  5.09745  4.48581  4.52562     4.6047   4.49192  4.67302   5.04594
 3.93195  2.81881  4.68835  5.20962  …  4.48509  4.51251  3.95692   5.20192
 3.16698  4.28198  5.098    5.05558     4.91201  4.86546  4.70301   4.33607
 3.83304  3.79805  4.37044  4.60972     5.16425  4.54111  3.88279   5.0377 
 5.6576   6.05616  5.06346  4.84959     5.03764  4.45284  5.83033   4.45421
 5.51183  2.49956  5.25686  4.71694     5.27198  5.04122  3.25039   4.65141
 2.74823  4.06677  4.79357  5.14666  …  4.96842  4.66686  3.84317   4.6201 
 1.21188  2.75412  4.94721  4.76064     4.76967  4.92002  2.35722   5.02438
 4.99028  5.29341  4.69187  4.42677     4.81216  5.16056  3.2374

In [5]:
using JuMP, Gurobi


function rspt(X,Y,rho,k,M,threshold)
    (l,p) = size(X)
    Xsquare = X.^2 
    Xlog= log.(X)
    Xsqrt = sqrt.(X)
    X = hcat(X,Xsquare,Xlog,Xsqrt)
    (n,m) = size(X)
    corrMat = cor(X)
    modelprob1 = Model(solver=GurobiSolver(OutputFlag=0))
#robustness with lasso penalty
    
@variable(modelprob1,b[1:m])
@variable(modelprob1,t)
@variable(modelprob1,theta)
@variable(modelprob1,z[1:m],Bin)
@variable(modelprob1,s[1:m])
#robustness
@constraint(modelprob1, sum(s[j] for j=1:m)<=theta)
for j = 1:m
    @constraint(modelprob1, b[j] <= s[j])
    @constraint(modelprob1, b[j] >= -s[j])
end
@constraint(modelprob1, norm(Y-X*b)<=t)

   
for j=1:m
        @constraint(modelprob1, b[j] <=  M * z[j])
        @constraint(modelprob1, b[j] >= -M * z[j])
end
    
#sparsity 
    @constraint(modelprob1, sum(z[j] for j=1:m)<=k)
#pairwise collinearity 
    pairwisexy = pairwise(corrMat,threshold) 
    (c,d)=size(pairwisexy)   
for i=1:c
    (d,e) = spitindices(pairwisexy,i)
    @constraint(modelprob1, z[e] + z[d]  <= 1)
end
    
    #NL transformation
     for i=1:p
         @constraint(modelprob1, z[i] + z[i+p] + z[i+2*p] + z[i+3*p] <= 1)
     end
        
@objective(modelprob1, Min, (t + rho*theta))

modelprob1
solve(modelprob1)
@show betastar = getvalue(b)


end


rspt (generic function with 1 method)

In [6]:
 function validmult(X)
    Xsquare = X.^2 
    Xlog= log.(X)
    Xsqrt = sqrt.(X)
    X = hcat(X,Xsquare,Xlog,Xsqrt)
    return X
end



validmult (generic function with 1 method)

In [7]:
#test
X = myData1[1:end-1]
Y = myData1[end]

betastar = rspt(Matrix(X),Y,.1,10,1,.7)

Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0, 0.380469, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.544994, 0.265614, 0.0, 0.0, 0.0, -0.00066829, 0.0, 0.0, 0.0, 0.00359103, 0.000972216, 0.000391949, -0.00397524, 0.00153658, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00360129, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


60-element Array{Float64,1}:
 0.0     
 0.0     
 0.0     
 0.380469
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.544994
 0.265614
 0.0     
 ⋮       
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     

In [8]:
function evaluate(X,y,β)
    norm(y - X * β)
end

evaluate (generic function with 1 method)

In [9]:
find(betastar)

10-element Array{Int64,1}:
  4
 11
 12
 16
 20
 21
 22
 23
 24
 30


Problem 1 , Dataset 1 ) Please run your algorithm on the datasets data1.csv to find the best regression
model you can. You will need to tune all of the hyperparameters in the algorithm to achieve the
best results. (In each dataset the y is the last column and the X is all but the last column).


In [29]:
X = myData2[1:end-1]
Y = myData2[end]
srand(1)
(train_X, train_Y), (test_X, test_Y) = splitobs(shuffleobs((X,Y)), at=.7)
(tr_X, tr_y), (vl_X, vl_y) = splitobs(shuffleobs((train_X, train_Y)), at=0.67)


tr_X = Matrix(tr_X)
vl_X = Matrix(vl_X)
train_X = Matrix(train_X)
test_X = Matrix(test_X)

best_rho = Inf
best_score = Inf
best_M = Inf
best_k = Inf
best_threshold = 1
for rho in [0.1, 1, 2]
    for M in [1, 2]
        for k in [5,7,10]
            for threshold in [.5,.7]
                Bstar = rspt(tr_X,tr_y,rho,k,M,threshold)
                score = evaluate(validmult(vl_X), vl_y, Bstar)
                @show rho,M,k,threshold,score
                if score < best_score
                best_score = score
                best_rho = rho
                best_M = M
                best_k = k
                best_threshold = threshold
                end
            end
        end
    end
end
@show best_rho, best_score, best_M, best_k, best_threshold



Academic license - for non-commercial use only
betastar = getvalue(b) = [0.933972, 0.614125, 0.0, 0.0, 0.0199671, 0.0, 0.0, 0.0, 0.0, 0.00614471, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.417282, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 5, 0.5, 1.2567077455193576)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.938869, 0.620156, 0.0542556, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00303672, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.404163, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 5, 0.7, 1.2358820220385252)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.933707, 0.61081, 0.0, 0.0, 0.0191555, 0.0148633, 0.0, 0.0, 0.0, 0.00623216, -0.00253449, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.416175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 7, 0.5, 1.2654962581721574)
Academic license - for non-commercial use only
betastar 

Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.152204, 0.151312, 0.113222, 0.0, 0.0361947, 0.0402564, 0.0379581, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 7, 0.5, 3.221685159350773)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0486015, 0.221831, 0.0, 0.0, 0.127344, 0.145129, 0.106922, 0.0, 0.0, 0.0349442, 0.0335081, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 7, 0.7, 2.8062062775876426)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.152204, 0.151312, 0.113222, 0.0, 0.0361947, 0.0402564, 0.0379581, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 10, 0.5, 3.221685159350773)
Academic license - for non-commercial use only
beta

(0.1, 1.2358820220385252, 1, 5, 0.7)

In [13]:
#TEST ERROR FOR DATA1

In [30]:
Bstar = rspt(tr_X,tr_y,best_rho,best_k,best_M,best_threshold)
score = evaluate(validmult(test_X), test_Y, Bstar)

Academic license - for non-commercial use only
betastar = getvalue(b) = [0.938869, 0.620156, 0.0542556, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00303672, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.404163, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.4480339448768456

Problem 1 Dataset 2) Please run your algorithm on the datasets data2.csv to find the best regression
model you can. You will need to tune all of the hyperparameters in the algorithm to achieve the
best results. (In each dataset the y is the last column and the X is all but the last column).

In [31]:
myData2

X = myData2[1:end-1]
Y = myData2[end]

using MLDataUtils
(train_X, train_Y), (test_X, test_Y) = splitobs(shuffleobs((X,Y)), at=.7)
(tr_X, tr_y), (v2_X, v2_y) = splitobs(shuffleobs((train_X, train_Y)), at=0.67)


tr_X = Matrix(tr_X)
v2_X = Matrix(v2_X)
test_X = Matrix(test_X)


best_rho = Inf
best_score = Inf
best_M = Inf
best_k = Inf
best_threshold = 1
for rho in [0.1, 1, 2]
    for M in [1, 2]
        for k in [5,7,10]
            for threshold in [.5,.7]
                Bstar = rspt(tr_X,tr_y,rho,k,M,threshold)
                score = evaluate(validmult(v2_X), v2_y, Bstar)
                @show rho,M,k,threshold,score
                if score < best_score
                best_score = score
                best_rho = rho
                best_M = M
                best_k = k
                best_threshold = threshold
                end
            end
        end
    end
end
@show best_rho, best_score, best_M, best_k, best_threshold



Academic license - for non-commercial use only
betastar = getvalue(b) = [0.929237, 0.586957, 0.0591415, 0.0, 0.0191637, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.427719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 5, 0.5, 1.267231556939014)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.929237, 0.586957, 0.0591415, 0.0, 0.0191637, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.427719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 5, 0.7, 1.267231556939014)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.930766, 0.589055, 0.0591774, 0.0, 0.0205218, 0.0, 0.0, 0.0, 0.0, 0.0, -0.000449689, 0.0, -0.000741861, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.427469, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (0.1, 1, 7, 0.5, 1.2708609155151813)
Academic license - for non-commercial use only
betastar =

Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0552015, 0.0, 0.0, 0.0, 0.150505, 0.149345, 0.10783, 0.0, 0.0367835, 0.0410529, 0.0323433, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 7, 0.5, 3.1429505756931415)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0552015, 0.0, 0.0, 0.0, 0.150505, 0.149345, 0.10783, 0.0, 0.0367835, 0.0410529, 0.0323433, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 7, 0.7, 3.1429505756931415)
Academic license - for non-commercial use only
betastar = getvalue(b) = [0.0, 0.0, 0.0552015, 0.0, 0.0, 0.0, 0.150505, 0.149345, 0.10783, 0.0, 0.0367835, 0.0410529, 0.0323433, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(rho, M, k, threshold, score) = (2.0, 1, 10, 0.5, 3.1429505756931415)
Academic license - for non-commercial u

(0.1, 1.267231556939014, 1, 5, 0.5)

In [17]:
#test error for data2

In [32]:
Bstar = rspt(tr_X,tr_y,best_rho,best_k,best_M,best_threshold)
score = evaluate(validmult(test_X), test_Y, Bstar)

Academic license - for non-commercial use only
betastar = getvalue(b) = [0.929237, 0.586957, 0.0591415, 0.0, 0.0191637, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.427719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.3229856966522116

In [ ]:
#Extra Credit : Implementation of bootstrapping method.= discussed in class.  

# using JuMP, Gurobi


# function ec(X,Y,rho,k,M,threshold)
#     (l,p) = size(X)
#     Xsquare = X.^2 
#     Xlog= log.(X)
#     Xsqrt = sqrt.(X)
#     X = hcat(X,Xsquare,Xlog,Xsqrt)
#     (n,m) = size(X)
#     corrMat = cor(X)
#     modelprob1 = Model(solver=GurobiSolver(OutputFlag=0))
# #robustness with lasso penalty
    
# @variable(modelprob1,b[1:m])
# @variable(modelprob1,t)
# @variable(modelprob1,theta)
# @variable(modelprob1,z[1:m],Bin)
# @variable(modelprob1,s[1:m])
# #robustness
# @constraint(modelprob1, sum(s[j] for j=1:m)<=theta)
# for j = 1:m
#     @constraint(modelprob1, b[j] <= s[j])
#     @constraint(modelprob1, b[j] >= -s[j])
# end
# @constraint(modelprob1, norm(Y-X*b)<=t)

   
# for j=1:m
#         @constraint(modelprob1, b[j] <=  M * z[j])
#         @constraint(modelprob1, b[j] >= -M * z[j])
# end
    
# #sparsity 
#     @constraint(modelprob1, sum(z[j] for j=1:m)<=k)
# #pairwise collinearity 
#     pairwisexy = pairwise(corrMat,threshold) 
#     (c,d)=size(pairwisexy)   
# for i=1:c
#     (d,e) = spitindices(pairwisexy,i)
#     @constraint(modelprob1, z[e] + z[d]  <= 1)
# end
    
#     #NL transformation
#      for i=1:p
#          @constraint(modelprob1, z[i] + z[i+p] + z[i+2*p] + z[i+3*p] <= 1)
#      end
        
# @objective(modelprob1, Min, (t + rho*theta))

# modelprob1
# solve(modelprob1)
# @show betastar = getvalue(b)
# ###### constraints given condition number per prof. bertsimas
# nonzerobeta=find(betastar)
#     e,f = size(nonzerobeta)
#     CNUM = cond(cor(X[:,find(betastar)]))
# maxiter = 3 
#     if CNUM >= 30 && iter < maxiter
#         iter = iter + 1
#         @constraint(modelprob1,sum(z[i] for i=1:e) <= k - 1)
#         solve(modelprob1)    
# end